### REPTE 3: Threshold Aggregation


In [8]:
import pandas as pd
import numpy as np
import cv2
import os
import glob
import random
import os
import glob
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torch.utils.data as data
import torchvision.transforms as transforms
import math
import itertools
from torch import nn as nn
import torch.nn.functional as F
from torch import Tensor
from numpy.matlib import repmat
import torch.optim as optim
from collections import OrderedDict
import cv2
import wandb
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
# save dicts to csv
import csv

In [2]:
import pandas as pd

# Cargar el CSV
csv_path = "patches_thresholds.csv"  # Asegúrate de que el nombre del archivo sea correcto
data = pd.read_csv(csv_path)

# Calcular la media de la columna 'threshold'
patches_mean_threshold = data['threshold'].mean()

print(f"La media de los valores en la columna 'threshold' es: {patches_mean_threshold}")

# Cargar el CSV
csv_path = "patient_thresholds.csv"  # Asegúrate de que el nombre del archivo sea correcto
data = pd.read_csv(csv_path)

# Calcular la media de la columna 'threshold'
patient_mean_threshold = data['threshold'].mean()

print(f"La media de los valores en la columna 'threshold' es: {patient_mean_threshold}")


La media de los valores en la columna 'threshold' es: 107.4
La media de los valores en la columna 'threshold' es: 0.20999999999999996


In [1]:
import pandas as pd

# Función para calcular la media ponderada
def weighted_mean(data, value_column, weight_column):
    weighted_sum = (data[value_column] * data[weight_column]).sum()
    total_weight = data[weight_column].sum()
    return weighted_sum / total_weight

# Rutas de los archivos
patches_csv_path = "patches_thresholds.csv"
patients_csv_path = "patient_thresholds.csv"
metrics_patches_path = "metrics_th1.csv"
metrics_patients_path = "metrics_th2.csv"

# Leer los archivos de thresholds
patches_data = pd.read_csv(patches_csv_path)
patients_data = pd.read_csv(patients_csv_path)

# Leer los archivos de métricas
metrics_patches = pd.read_csv(metrics_patches_path)
metrics_patients = pd.read_csv(metrics_patients_path)

# Verificar que existan las columnas necesarias
required_columns_thresholds = {'fold', 'threshold'}
required_columns_metrics = {'fold', 'recall'}
if not required_columns_thresholds.issubset(patches_data.columns) or not required_columns_thresholds.issubset(patients_data.columns):
    raise ValueError(f"Los CSV de thresholds deben contener las columnas: {required_columns_thresholds}")
if not required_columns_metrics.issubset(metrics_patches.columns) or not required_columns_metrics.issubset(metrics_patients.columns):
    raise ValueError(f"Los CSV de métricas deben contener las columnas: {required_columns_metrics}")

# Combinar datos de thresholds con sus respectivas métricas usando la columna 'fold' como clave
patches_combined = pd.merge(patches_data, metrics_patches, on='fold', how='inner')
patients_combined = pd.merge(patients_data, metrics_patients, on='fold', how='inner')

# Calcular la media ponderada para cada conjunto de datos
patches_weighted_mean = weighted_mean(patches_combined, 'threshold', 'recall')
patients_weighted_mean = weighted_mean(patients_combined, 'threshold', 'recall')

# Imprimir los resultados
print(f"La media ponderada de los thresholds en 'patches_thresholds.csv' es: {patches_weighted_mean:.4f}")
print(f"La media ponderada de los thresholds en 'patient_thresholds.csv' es: {patients_weighted_mean:.4f}")


La media ponderada de los thresholds en 'patches_thresholds.csv' es: 107.3081
La media ponderada de los thresholds en 'patient_thresholds.csv' es: 0.2142


In [38]:
import pandas as pd
import numpy as np


def calcular_media(data):
    return data['threshold'].mean()

def calcular_mediana(data):
    return data['threshold'].median()

def calcular_desviacion_estandar(data):
    return data['threshold'].std()

def calcular_minimo(data):
    return data['threshold'].min()

def calcular_maximo(data):
    return data['threshold'].max()

def calcular_cuartiles(data):
    q1 = data['threshold'].quantile(0.25)
    q3 = data['threshold'].quantile(0.75)
    return q1, q3

def calcular_varianza(data):
    return data['threshold'].var()

def resumen_descriptivo(data):
    return data['threshold'].describe()

def normalizar(data):
    min_val = data['threshold'].min()
    max_val = data['threshold'].max()
    data['normalized_threshold'] = (data['threshold'] - min_val) / (max_val - min_val)
    return data

def calcular_embedding(data):
    return np.mean(data['threshold'].values)

In [39]:
data = pd.read_csv('patches_thresholds.csv')

media = calcular_media(data)
mediana = calcular_mediana(data)
std_dev = calcular_desviacion_estandar(data)
q1, q3 = calcular_cuartiles(data)
varianza = calcular_varianza(data)
normalizado = normalizar(data)
embedding = calcular_embedding(data)
minimo = calcular_minimo(data)
maximo = calcular_maximo(data)

print(f"Media: {media}")
print(f"Mediana: {mediana}")
print(f"Minimo: {minimo}")
print(f"Maximo: {maximo}")
#print(f"Desviación estándar: {std_dev}")
print(f"Cuartiles: Q1={q1}, Q3={q3}")
#print(f"Varianza: {varianza}")
#print("Datos normalizados:")
#print(normalizado)
print(embedding)


Media: 107.4
Mediana: 107.0
Minimo: 101
Maximo: 113
Cuartiles: Q1=104.0, Q3=112.0
107.4


In [56]:
import pandas as pd

# Cargar el archivo CSV
csv_path = "metrics_th2.csv"  # Cambia al nombre de tu archivo
data = pd.read_csv(csv_path, delimiter=",")  # Usa `delimiter="\t"` si es TSV


# Calcular el rango (mínimo y máximo) para cada columna de métricas
def calcular_rangos(data):
    rangos = {}
    #print(data)
    #print(data.columns)
    for columna in data.columns:  # Excluir la columna `fold`
        minimo = data[columna].min()
        maximo = data[columna].max()
        #print(minimo, maximo)
        rangos[columna] = (minimo, maximo)
    return rangos

# Llamar a la función y mostrar resultados
rangos = calcular_rangos(data)
#print(rangos)

# Imprimir los rangos
for metrica, (min_val, max_val) in rangos.items():
    print(f"Métrica: {metrica}")
    print(f"  Mínimo: {min_val}")
    print(f"  Máximo: {max_val}")
    print("---")

Métrica: fold
  Mínimo: 0
  Máximo: 4
---
Métrica: accuracy
  Mínimo: 0.2903225806451613
  Máximo: 0.7096774193548387
---
Métrica: precision
  Mínimo: 0.3
  Máximo: 0.7333333333333333
---
Métrica: recall
  Mínimo: 0.3333333333333333
  Máximo: 0.6875
---
Métrica: f1_score
  Mínimo: 0.3157894736842105
  Máximo: 0.7096774193548386
---
Métrica: tp
  Mínimo: 3
  Máximo: 11
---
Métrica: tn
  Mínimo: 3
  Máximo: 15
---
Métrica: fp
  Mínimo: 4
  Máximo: 12
---
Métrica: fn
  Mínimo: 5
  Máximo: 10
---
